# Information Extraction - Assignment
This assignment is based on the Information Extraction lecture and the lab.


In [53]:
import nltk
import json
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import re
from statistics import mode
from google.colab import files
import io

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [54]:
#inputfile='../football_players.txt' #Location of the file
#buf=open(inputfile, encoding="UTF-8")
#list_of_doc=buf.read().split('\n')

uploaded = files.upload()
file = io.StringIO(uploaded['football_players.txt'].decode('utf-8')).read().split('\n')
# deletes any spaces 
list_of_doc = [x for x in file if x !='\r']
print(list_of_doc[0])

l = []
# remove "" in document, as they alter the ne_chunk and POS tagging and make them return wrong results
for i in list_of_doc:
    for k in i:
        if k == '"':
            i = i.replace('"', '')
    if len(i) != 0:
        l.append(i)
        
list_of_doc = l

# setting up lookups to be refer to in certain code sections
countries = ['Spain', 'England', 'Portugal', 'Wales', 'England', 'Brazil', 'Germany', 'Argentina', 'Sweden']
clubs = ['Manchester United', "Barcelona FC", "FC", "Real",  "Real Madrid"]
nationalities = ["English", "Spanish", "Portuguese", "Irish", "Swedish", 'English', "Brazilian", "German"]
position_list = ["forward", "goalkeeper", "left-back", "right-back", "defensive midfield", "centre back", "sweeper",
                     "wing back", "centre midfield", "attacking midfield", "striker", "midfield", "central midfield", "left midfield",
                     "right midfield", "right winger", "winger"]
months_of_year = ["Janruary", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

Saving football_players.txt to football_players (1).txt
Cristiano Ronaldo dos Santos Aveiro, ComM, GOIH (born 5 February 1985) is a Portuguese professional footballer who plays for Spanish club Real Madrid and the Portugal national team. He is a forward and serves as captain for Portugal. In 2008, he won his first Ballon d'Or and FIFA World Player of the Year awards. He then won the FIFA Ballon d'Or in 2013 and 2014. In 2015, Ronaldo scored his 500th senior career goal for club and country. Often ranked as the best player in the world, Ronaldo was named the best Portuguese player of all time by the Portuguese Football Federation, during its 100th anniversary celebrations in 2015. He is the only player to win four European Golden Shoe awards. One of the most marketable athletes in sport, in 2016 Forbes named Ronaldo the world's best paid athlete. In June 2016, ESPN ranked him the world's most famous athlete. Ronaldo began his club career playing for Sporting CP, before signing with Manc

# Task 1 (10 Marks)
Write a function that takes each document and performs:
1) sentence segmentation 2) tokenization 3) part-of-speech tagging

Please keep in mind that the expected output is a list within a list as shown below.


In [0]:
def ie_preprocess(documents):

    pos_sentences = nltk.sent_tokenize(documents)
    pos_sentences = [nltk.word_tokenize(sent) for sent in pos_sentences]
    pos_sentences = [nltk.pos_tag(sent) for sent in pos_sentences]
    return pos_sentences



Run the following code to check your result for the first document (Ronaldo).

In [56]:
first_doc=list_of_doc[0]
pos_sent=ie_preprocess(first_doc)
pos_sent

[[('Cristiano', 'NNP'),
  ('Ronaldo', 'NNP'),
  ('dos', 'NN'),
  ('Santos', 'NNP'),
  ('Aveiro', 'NNP'),
  (',', ','),
  ('ComM', 'NNP'),
  (',', ','),
  ('GOIH', 'NNP'),
  ('(', '('),
  ('born', 'VBN'),
  ('5', 'CD'),
  ('February', 'NNP'),
  ('1985', 'CD'),
  (')', ')'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('Portuguese', 'JJ'),
  ('professional', 'JJ'),
  ('footballer', 'NN'),
  ('who', 'WP'),
  ('plays', 'VBZ'),
  ('for', 'IN'),
  ('Spanish', 'JJ'),
  ('club', 'NN'),
  ('Real', 'NNP'),
  ('Madrid', 'NNP'),
  ('and', 'CC'),
  ('the', 'DT'),
  ('Portugal', 'NNP'),
  ('national', 'JJ'),
  ('team', 'NN'),
  ('.', '.')],
 [('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')],
 [('In', 'IN'),
  ('2008', 'CD'),
  (',', ','),
  ('he', 'PRP'),
  ('won', 'VBD'),
  ('his', 'PRP$'),
  ('first', 'JJ'),
  ('Ballon', 'NNP'),
  ("d'Or", 'NN'),
  ('and', 'CC')

Expected output
 [...[('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')], ...]

# Task 2 (20 Marks)
Write a function that will take the list of tokens with POS tags for each sentence and returns the named entities (NE). 

Hint: Use binary=True while calling NE chunk function

In [0]:
#Expanded the Named_Entity_Finding function to be able to take in binary false and true, and also specify NE or person etc
def named_entity_finding(pos_sent, x , y):  # Code goes here
    named_entities = []
    if x is True:
        tree = nltk.ne_chunk(pos_sent, binary=True)
        last_value = ""
        for subtree in tree.subtrees():
            if subtree.label() == 'NE':
                entity = ""
                for leaf in subtree.leaves():
                    entity = entity + leaf[0] + " "
                named_entities.append(entity.strip())
        return named_entities

    elif x is False and y == "All":
        tree = nltk.ne_chunk(pos_sent, binary=False)
        for subtree in tree.subtrees():
            if subtree.label() == 'PERSON':
                entity = ""
                for leaf in subtree.leaves():
                    entity = entity + leaf[0] + " "
                    named_entities.append(entity.strip())

            elif subtree.label() == 'ORGANIZATION':
                    entity = ""
                    for leaf in subtree.leaves():
                        entity = entity + leaf[0] + " "
                        named_entities.append(entity.strip())

            elif subtree.label() == 'GPE':
                    entity = ""
                    for leaf in subtree.leaves():
                        entity = entity + leaf[0] + " "
                        named_entities.append(entity.strip())

    elif x is True and y == "NE":
        tree = nltk.ne_chunk(pos_sent, binary=False)
        for subtree in tree.subtrees():
            if subtree.label() == 'NNP':
                for leaf in subtree.leaves():
                    if leaf[1] == 'NNP':
                        named_entities.append(leaf[0])
    return named_entities


In [58]:
pos_sents=ie_preprocess(first_doc)
named_entity_finding(pos_sents[0], True, "")

['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

Expected output ['Cristiano Ronaldo', 'Santos Aveiro', 'ComM', 'GOIH', 'Portuguese', 'Spanish', 'Real Madrid', 'Portugal']

# Task 3 (10 Marks)

Now use the named_entity_finding() function to extract all NEs for each document.

Hint: pos_sents holds the list of lists of tokens with POS tags

In [0]:
def NE_flat_list_fn(pos_sents):
    NE = []
    for pos_sent in pos_sents:
        pos_sent = ie_preprocess(pos_sent)
        for pos in pos_sent:
            named_ents = named_entity_finding(pos, True, "")
            NE.append(named_ents)
        NE_flat_list = [val for sublist in NE for val in sublist]
        NE_flat_list = list(NE_flat_list)
    return NE_flat_list




In [60]:
print(NE_flat_list_fn(list_of_doc))

['Cristiano Ronaldo', 'Santos Aveiro', 'ComM', 'GOIH', 'Portuguese', 'Spanish', 'Real Madrid', 'Portugal', 'Portugal', 'Ballon', 'FIFA', 'FIFA Ballon', 'Ronaldo', 'Ronaldo', 'Portuguese', 'Portuguese Football Federation', 'European Golden Shoe', 'ESPN', 'Ronaldo', 'Manchester United', 'England', 'United', 'UEFA Champions League', 'FIFA Club', 'Ballon', 'FIFA', 'Manchester United', 'Madrid', 'Spain', 'Ronaldo', 'UEFA Champions League', 'Ronaldo', 'La Liga', 'Ronaldo', 'UEFA Champions League', 'Real Madrid', 'La Liga', 'Lionel Messi', 'Ronaldo', 'Portugal', 'Portugal', 'European', 'FIFA World Cups', 'Portuguese', 'Portugal', 'Portugal', 'Portugal', 'Ronaldo', 'UEFA European', 'European', 'Michel Platini', 'Ronaldo', 'Portugal', 'France', 'Silver Boot', 'Lionel Andrés Leo Messi', 'Spanish', 'Argentina', 'Messi', 'FIFA Ballons', 'European Golden Shoes', 'Messi', 'La Liga', 'La Liga', 'Copa América', 'Born', 'Argentina', 'Messi', 'Spain', 'Barcelona', 'Barcelona', 'Messi', 'Ballon', 'FIFA',

# Task 4 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the re.compile() function to create the extraction patterns

Reference: https://docs.python.org/3/howto/regex.html

In [0]:
def name_of_the_player(doc):
    x = 0
    k = 0
    names_out = []
    doc = ie_preprocess(doc)
    for d in doc:
        if x == "break":
            return names_out

        name_list = named_entity_finding(d, True, "NE")
        name_holder = " ".join(name_list)
        name_holder = name_holder.rstrip()
        name_holder = name_holder.lstrip()
        name_split = name_holder.split(" ")
        names = []
        for j in name_split:
            names.append(j)
            if k == 3:
                break
            k = k + 1

        for name in names:
            if name in countries:
                names.remove(name)
            elif name in clubs:
                names.remove(name)
            elif name in nationalities:
                names.remove(name)

        for name in names:
            if name in countries:
                names.remove(name)
            elif name in clubs:
                names.remove(name)
            elif name in nationalities:
                names.remove(name)

        seen = set()
        names = [x for x in names if not (x in seen or seen.add(x))]
        names_out = " ".join(names)
        x == "break"
        return names_out


In [62]:
print(name_of_the_player(first_doc))

Cristiano Ronaldo Santos Aveiro


In [0]:
def country_of_origin(doc):
    coo_out = []
    r1 = re.compile('is a\s+((?:[\S,]+\s+))professional footballer')
    coo1 = r1.findall(doc)

    r2 = re.compile(r'is a\s+((?:[\S,]+\s+))footballer')
    coo2 = r2.findall(doc)

    r3 = re.compile(r'is a\s+((?:[\S,]+\s+))former professional footballer')
    coo3 = r3.findall(doc)

    r4 = re.compile(r'the\s+((?:[\S,]+\s+))national team')
    coo4 = r4.findall(doc)

    if re.search(r1, doc):
        coo = coo1
    elif re.search(r2, doc):
        coo = coo2
    elif re.search(r3, doc):
        coo = coo3
    elif re.search(r4, doc):
        coo = coo4

    coo_out = " ".join(coo)
    return coo_out

In [64]:
print(country_of_origin(first_doc))

Portuguese 


In [0]:
def date_of_birth(doc):
    dob_sentences = nltk.word_tokenize(doc)
    r = re.compile(r'\(born (.*?)\)')
    date = r.findall(doc)
    date_out = " ".join(date)
    return date_out

In [66]:
print(date_of_birth(first_doc))

5 February 1985


In [0]:
def team_of_the_player(doc):
    team_hold = []
    team = []
    teams = []
    docs = nltk.sent_tokenize(doc)
    docs_word = nltk.word_tokenize(doc)
    j = 0
    for doc in docs_word:
      # if player is currently not playing for a team, "played" is invalid for them, then they are retired.
        if doc == "plays" or doc == "retired":
            j = j+1
    if j == 0:
        teams.append("Retired")
        teams_out = " ".join(teams)
        return teams_out
    all_ne = []
    i = 0
    for d in docs:
        if i == 2:
            break
        else:
            d_pos = nltk.word_tokenize(d)
            d_pos = nltk.pos_tag(d_pos)
            ne_finding = named_entity_finding(d_pos, False, "All")
            all_ne.append(ne_finding)

            national_team = re.compile(r'((?:[\S,]+\s+){0,1})national team')
            national_team = national_team.findall(d)
            team_hold.append(national_team)

            club_team1 = re.compile('club\s+((?:[\S,]+\s*){0,1})')
            club_team = club_team1.findall(d)
            if re.search(club_team1, d):
                for club in club_team:
                    if club.rstrip() == "FC" or club.rstrip() == "Manchester" or club.rstrip() == "Real":
                        club_team = re.compile('club\s+((?:[\S,]+\s*){0,2})')
                        club_team = club_team.findall(d)
                else:
                    team_hold.append(club_team)

            club_team2 = re.compile(r'for\s+((?:[\S,]+\s*){0,1}).')
            club_team_ = club_team2.findall(d)
            if re.search(club_team2, d):
                for club in club_team_:
                    if club.rstrip() == "FC" or club.rstrip() == "Manchester" or club.rstrip() == "Real":
                        club_team2 = re.compile('for\s+((?:[\S,]+\s*){0,2}).')
                        club_team_ = club_team2.findall(d)
                        team_hold.append(club_team_)


        i = i + 1
    team_hold = [x for x in team_hold if x]

    for sublist in team_hold:
        for item in sublist:
            team.append(item.rstrip())

    all_ne_c = []
    for sublist in all_ne:
        for item in sublist:
            all_ne_c.append(item)

    for item in team:
        if item in all_ne_c:
            teams.append(item)
    teams = set(teams)
    teams_out = ", ".join(teams)
    return teams_out

In [68]:
print(team_of_the_player(first_doc))

Portugal, Real Madrid


In [0]:
def position_of_the_player(doc): #code goes here
    # code goes here
    position_array = []
    position_out = []
    for position in position_list:
        r = re.compile(position)
        find = r.findall(doc)
        position_array.append(find)

    position_array = [x for x in position_array if x != []]

    for position in position_array:
        position = set(position)
        position_out.append(position)

    position_array = [x for x in position_array if x]
    if len(position_array) == 0:
        position_array.append("Position Data Not Available")

    position_out = [val for sublist in position_out for val in sublist]
    return position_out

In [70]:
print(position_of_the_player(first_doc))

['forward']


# Task 5 (10 Marks)

Write a function using the outputs from the previous functions to generate JSON-LD output as follows.

Reference: https://json-ld.org/primer/latest/

{ "@id": "http://my-soccer-ontology.com/footballer/name_of_the_player",

    "name": "",
    "born": "",
    "country": "",
    "position": [
        { "@id": "http://my-soccer-ontology.com/position",
            "type": ""
        }
     ]   
     "team": [
        { "@id": "http://my-soccer-ontology.com/team",
            "name": ""
        }   
     ]
}


In [73]:
def generator(doc):
    arg_name = name_of_the_player(doc)
    arg_name = arg_name.replace("ê","e").replace("é","e").replace("ú","u").replace("ć","c").replace("ö","o").replace("Ö","O")
    arg_club = team_of_the_player(doc)
    arg_club = arg_club.replace("ê","e").replace("é","e").replace("ú","u").replace("ć","c").replace("ö","o").replace("Ö","O")
    
    holder = [arg_name, date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), arg_club]
    return holder


holder = generator(list_of_doc[0])
print(holder)


['Cristiano Ronaldo Santos Aveiro', '5 February 1985', 'Portuguese ', ['forward'], 'Portugal, Real Madrid']


In [0]:
def generate_jsonId(data):
    ld = {"@id": "http://my-soccer-ontology.com/footballer/" + data[0],

          "name": data[0],
          "born": data[1],
          "country": data[2],
          "position": [
              {"@id": "http://my-soccer-ontology.com/position/",
               "type": data[3]
               }
          ],
          "team": [
              {"@id": "http://my-soccer-ontology.com/team/",
               "name": data[4]
               }
          ]
          }

    return json.dumps(ld)
  

In [75]:
print(generate_jsonId(holder))

{"@id": "http://my-soccer-ontology.com/footballer/Cristiano Ronaldo Santos Aveiro", "name": "Cristiano Ronaldo Santos Aveiro", "born": "5 February 1985", "country": "Portuguese ", "position": [{"@id": "http://my-soccer-ontology.com/position/", "type": ["forward"]}], "team": [{"@id": "http://my-soccer-ontology.com/team/", "name": "Portugal, Real Madrid"}]}


# Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract this. Also extend the JSON-LD output accordingly.

In [0]:
def international_club_debut(doc):
    club_holder = []
    club_debut = []
    international_debut = []
    international_holder = []

    numeric = re.compile('\d{4}')
    r1 = re.compile('international debut\s+((?:[\S,]+\s*){0,6})')
    debut1 = r1.findall(doc)

    r2 = re.compile('club debut\s+((?:[\S,]+\s*){0,6})')
    debut2 = r2.findall(doc)

    r3 = re.compile('professional debut\s+((?:[\S,]+\s*){0,6})')
    debut3 = r3.findall(doc)

    r4 = re.compile('competitive debut\s+((?:[\S,]+\s*){0,6})')
    debut4 = r4.findall(doc)

    r5 = re.compile('England debut on\s+((?:[\S,]+\s*){0,10})')
    debut5 = r5.findall(doc)

    r6 = re.compile('first-team debut\s+((?:[\S,]+\s*){0,8})')
    debut6 = r6.findall(doc)

    if re.search(r1, doc):
        international_debut = debut1
    if re.search(r2, doc):
        club_debut = debut2
    if re.search(r3, doc):
        club_debut = debut3
    if re.search(r4, doc):
        club_debut = debut4
    if re.search(r5, doc):
        international_debut = debut5
    if re.search(r6, doc):
        club_debut = debut6

    if re.search(r1, doc) or re.search(r5, doc):
        for international_tokens in international_debut:
            international_tokens = nltk.word_tokenize(international_tokens)
        each_international= []
        for each in international_tokens:
            each_international.append(each.rstrip())
        for word in each_international:
            if re.findall(numeric, word):
                international_holder.append(word)

    if len(international_holder) == 0:
        international_holder.append(("NA"))

    if re.search(r2, doc) or re.search(r3, doc) or re.search(r4, doc) or re.search(r6, doc):
        for club_tokens in club_debut:
            club_tokens = nltk.word_tokenize(club_tokens)
        each_club = []
        for each in club_tokens:
            each_club.append(each.rstrip())
        for word in each_club:
            if re.findall(numeric, word):
                club_holder.append(word)

    if len(club_holder) == 0:
        club_holder.append(("NA"))

    return "International Debut Year:",international_holder,"Club Debut Year:",club_holder

In [79]:
def generator2(doc):
  
    arg_name = name_of_the_player(doc)
    arg_name = arg_name.replace("ê","e").replace("é","e").replace("ú","u").replace("ć","c").replace("ö","o").replace("Ö","O")
    arg_club = team_of_the_player(doc)
    arg_club = arg_club.replace("ê","e").replace("é","e").replace("ú","u").replace("ć","c").replace("ö","o").replace("Ö","O")
    
    holder = [arg_name, date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), arg_club ,international_club_debut(doc) ]
    return holder


holder2 = generator2(list_of_doc[0])
print(holder2)

['Cristiano Ronaldo Santos Aveiro', '5 February 1985', 'Portuguese ', ['forward'], 'Portugal, Real Madrid', ('International Debut Year:', ['2003'], 'Club Debut Year:', ['NA'])]


In [0]:
def generate_jsonId2(data):
    ld = {"@id": "http://my-soccer-ontology.com/footballer/" + data[0],

          "name": data[0],
          "born": data[1],
          "country": data[2],
          "position": [
              {"@id": "http://my-soccer-ontology.com/position/",
               "type": data[3]
               }
          ],
          "team": [
              {"@id": "http://my-soccer-ontology.com/team/",
               "name": data[4],
               "debut" : data[5]
               }
          ]
          }

    return json.dumps(ld)

In [81]:
print(generate_jsonId2(holder2))

{"@id": "http://my-soccer-ontology.com/footballer/Cristiano Ronaldo Santos Aveiro", "name": "Cristiano Ronaldo Santos Aveiro", "born": "5 February 1985", "country": "Portuguese ", "position": [{"@id": "http://my-soccer-ontology.com/position/", "type": ["forward"]}], "team": [{"@id": "http://my-soccer-ontology.com/team/", "name": "Portugal, Real Madrid", "debut": ["International Debut Year:", ["2003"], "Club Debut Year:", ["NA"]]}]}
